In [11]:
from langchain_groq import ChatGroq
from CONFIG import GROQ_MODEL
from __future__ import annotations
from pydantic import BaseModel, Field
from typing import TypedDict, List, Annotated
from langchain_core.messages import AIMessage, SystemMessage, AnyMessage, HumanMessage
from dotenv import load_dotenv
import operator
from langgraph.types import Send

In [7]:
load_dotenv()
llm = ChatGroq(model=GROQ_MODEL)

In [ ]:
class TASK(BaseModel):
    id: int
    title: str
    brief: str = Field(..., description='what to cover')

class PLAN(BaseModel):
    title: str
    tasks: List[TASK]

class state(TypedDict):
    topic: str
    plan: PLAN
    sections: Annotated[List[str], operator.add]
    final_blog: str

In [10]:
def orchestrator(state: state):
    plans = llm.with_structured_output(
        PLAN
    ).invoke(
        SystemMessage(
            content='generate blog plan with 5-7 different but related sections on the following TOPIC'
        ),
        HumanMessage(
            content=f"Topic is: {state['topic']}"
	  )
    )
    return {"plan": PLAN}

In [ ]:
def fanout(state: state):
    return [
        Send(
            'worker',
            {
                'task': task,
                'topic': state['topic'],
                'plan': state['plan']
            }
        )
        for task in state['plan'].tasks
    ]